# Shallow Profiler: Technical


This document documents the code.



From the imported `shallowprofiler.py` module we have available `sensors`: A list of lists. 
Each entry is `[sensor_name, instrument_name]` so for example `sensors[4]` is `['temperature', 'ctd']`.
To see the entirety run a Python cell with just `sensors`.

With instrument and sensor names sorted (with spectral irradiance a special case) the next step
is to build a data dictionary called **`d `** with keys corresponding to sensor (data) types:
**`conductivity`, `temperature`, `pco2`** etcetera. The corresponding value for each sensor
key is a five-tuple indexed \[0\], \[1\], ..., \[4\].


```
0: XArray DataArray: sensor data
1: XArray DataArray: sensor depth (meters, negative down) corresponding to data
2: float: Default charting lower limit for data
3: float: Default charting upper limit for data
4: string: Default chart color e.g. "blue"
```